In [5]:
import mne

dataset1 = mne.read_epochs("./beta_epo1.fif")
dataset2 = mne.read_epochs("./beta_epo2.fif")
# entendimento dos dados
dataset1.get_data().shape, dataset2.get_data().shape

Reading /Users/uiuizap/Documents/facul/patter-recog/SSVEP/beta/beta_epo1.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    2996.00 ms
        0 CTF compensation matrices available
Not setting metadata
160 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/uiuizap/Documents/facul/patter-recog/SSVEP/beta/beta_epo2.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    2996.00 ms
        0 CTF compensation matrices available
Not setting metadata
160 matching events found
No baseline correction applied
0 projection items activated


/var/folders/5y/hp3j720d4tld59t5jpzjfdg00000gn/T/ipykernel_8976/3089845198.py:3: RuntimeWarning: This filename (./beta_epo1.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  dataset1 = mne.read_epochs("./beta_epo1.fif")
/var/folders/5y/hp3j720d4tld59t5jpzjfdg00000gn/T/ipykernel_8976/3089845198.py:4: RuntimeWarning: This filename (./beta_epo2.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  dataset2 = mne.read_epochs("./beta_epo2.fif")


((160, 8, 750), (160, 8, 750))

In [6]:
import numpy as np

hits_dataset1 = 0
hits_dataset2 = 0

y_dataset1 = np.load("../datasets/beta/labels1.npy")
y_dataset2 = np.load("../datasets/beta/labels2.npy")

threshold = 0.20
targets = [float(item) for item in dataset1.event_id.keys()]

for i in range(len(dataset1)):
    view_dataset1 = dataset1[i].compute_psd(method='welch', fmin=7, fmax=17, verbose=False)
    view_dataset2 = dataset2[i].compute_psd(method='welch', fmin=7, fmax=17, verbose=False)

    for target in targets:
        fmin = target - threshold
        fmax = target + threshold

        # Dataset 1
        features_dataset1 = view_dataset1.get_data(fmin=fmin, fmax=fmax)
        X_dataset1 = np.array(features_dataset1)
        max_frequency_dataset1 = np.max(X_dataset1, axis=1)

        if np.any((max_frequency_dataset1 >= fmin) & (max_frequency_dataset1 <= fmax)):
            hits_dataset1 += 1

        # Dataset 2
        features_dataset2 = view_dataset2.get_data(fmin=fmin, fmax=fmax)
        X_dataset2 = np.array(features_dataset2)
        max_frequency_dataset2 = np.max(X_dataset2, axis=1)

        if np.any((max_frequency_dataset2 >= fmin) & (max_frequency_dataset2 <= fmax)):
            hits_dataset2 += 1

accuracy_dataset1 = (hits_dataset1 / len(y_dataset1)) * 100 
accuracy_dataset2 = (hits_dataset2  / len(y_dataset2)) * 100

print(f"Accuracia dataset 1: {accuracy_dataset1} %")
print(f"Accuracia dataset 2: {accuracy_dataset2} %")

Accuracia dataset 1: 0.625 %
Accuracia dataset 2: 10.0 %
